## experiment 2 (flare search)
* here we are looking for samples that are in a main category but not in the child ones
* an exmaple is samples that are in anthrohony but not in aircraft
* here we did not filter aircrafts with bio and bird because there was not enough sample otherwise

In [1]:
import os 
import importlib

path='/home/enis/projects/nna/src/scripts/'
os.chdir(path)
import teacher

import itertools
import pandas as pd

In [17]:
aa=[('dalton', '01'),
        ('dalton', '02'),
        ('dalton', '03'),
        ('dalton', '04'),
        ('dalton', '05'),
        ('dalton', '06'),
        ('dalton', '07'),
        ('dalton', '08'),
        ('dalton', '09'),
        ('dalton', '10')
    ] + [('dempster', '11'), ('dempster', '12'), ('dempster', '13'),
         ('dempster', '14'), ('dempster', '15'), ('dempster', '16'),
         ('dempster', '17'), ('dempster', '19'), ('dempster', '20'),
         ('dempster', '21'), ('dempster', '22'), ('dempster', '23'),
         ('dempster', '24'), ('dempster', '25')] + [('ivvavik', 'AR01'),
                                                    ('ivvavik', 'AR02'),
                                                    ('ivvavik', 'AR03'),
                                                    ('ivvavik', 'AR04'),
                                                    ('ivvavik', 'AR05'),
                                                    ('ivvavik', 'AR06'),
                                                    ('ivvavik', 'AR07'),
                                                    ('ivvavik', 'AR08'),
                                                    ('ivvavik', 'AR09'),
                                                    ('ivvavik', 'AR10'),
                                                    ('ivvavik', 'SINP01'),
                                                    ('ivvavik', 'SINP02'),
                                                    ('ivvavik', 'SINP03'),
                                                    ('ivvavik', 'SINP04'),
                                                    ('ivvavik', 'SINP05'),
                                                    ('ivvavik', 'SINP06'),
                                                    ('ivvavik', 'SINP07'),
                                                    ('ivvavik', 'SINP08'),
                                                    ('ivvavik', 'SINP09'),
                                                    ('ivvavik', 'SINP10')]

In [18]:
len(aa)

44

In [14]:
a=['41', '21', '49', '48', '19', '16', '39', '30', '38', '22', '11', '37',
        '44', '33', '29', '46', '25', '13', '24', '17', '40', '14']+ ['15', '20', '31', '47', '34']+['12', '27', '26', '45', '50', '18', '32', '36']

In [15]:
sorted(a)

['11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '24',
 '25',
 '26',
 '27',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50']

In [ ]:
import glob
from pathlib import Path


In [144]:
importlib.reload(teacher)
config = teacher.setup()

## data-1: as many sampes as possible
config['new_dataset_path'] = './datasetV2.1.2.csv'
config['dataset_version'] = 'V2.1.2'
config['versiontag'] = '3rk9ayjc-V1'

# try! to collect same amount of samples from each location
config['balance_by_location'] = False
config['confidence_thresholds'] = {
        'biophony':0.99, 
            'insect': 0.99,
            'bird':0.99,
                'songbirds':0.95,
                'duck-goose-swan': 0.9,
                'grouse-ptarmigan': 0.9,
        'anthrophony': 0.8,
            'aircraft': 0.99,
        'silence': 0.99,
    }

# only applies to main label
config['labels_thresholds'] = {
    'default': 0.5,
    'biophony': 0.9,
    'insect': 0.5,
    'bird': 0.9,
    'songbirds': 0.9,
    'duck-goose-swan': 0.5,
    'grouse-ptarmigan': 0.5,
    'anthrophony': 0.08,
    'aircraft': 0.5,
    'silence': 0.5,
}

# I DID NOT USE THESE NUMBERS
config['new_data_classes_counts'] = {
    'biophony':10*3,
        'insect': 10*4,
        'bird':10**3,
            'songbirds':10*4,
            'duck-goose-swan': 10*4,
            'grouse-ptarmigan': 10*4,
    'anthrophony': 10*4,
        'aircraft': 10**4,
    'silence': 10*3,
}




confident_preds_dict = {}
# config['new_data_classes_counts'].keys()
for label_to_sample in ['bird','biophony']:
    print("--------",label_to_sample)
    confidence_threshold=config['confidence_thresholds'][label_to_sample]
    confident_preds = teacher.pick_confidents(preds_df, label_to_sample,
                                                     confidence_threshold,
                                                     config['related_cols'],
                                                     config['labels_thresholds'],
                                                    columns_to_exclude=['TIMESTAMP','sum','location','region'])
    confident_preds_dict[label_to_sample] = confident_preds.copy()

selected_data={}

# sort values by sum so other lables has lower confidence
for k,v in confident_preds_dict.items():
    confident_preds_dict[k]=v.sort_values(by='sum', ascending=True)
    
#select samples
for k,v in confident_preds_dict.items():
    print(k,len(v))
    if k in ['biophony', 'bird']:
        selected_data[k]=v.iloc[:50000]
    else:
        selected_data[k]=v.iloc[:2000]

only_biophony=(confident_preds_dict['biophony'][~confident_preds_dict['biophony'].isin( confident_preds_dict['bird']).all(1)])
only_biophony=only_biophony.sort_values(by='bird', ascending=True)
only_biophony=only_biophony[only_biophony['bird']<=0.112]

-------- bird
anthrophony
1099053
aircraft
1099053
insect
1081408
silence
1056244
-------- biophony
anthrophony
1998831
aircraft
1998831
insect
1732830
silence
1699232
bird 1056244
biophony 1699232


In [121]:
importlib.reload(teacher)
config = teacher.setup()

## data-1: as many sampes as possible
config['new_dataset_path'] = './datasetV2.1.2.csv'
config['dataset_version'] = 'V2.1.2'
config['versiontag'] = '3rk9ayjc-V1'

# try! to collect same amount of samples from each location
config['balance_by_location'] = False
config['confidence_thresholds'] = {
        'biophony':0.99, 
            'insect': 0.99,
            'bird':0.99,
                'songbirds':0.95,
                'duck-goose-swan': 0.9,
                'grouse-ptarmigan': 0.9,
        'anthrophony': 0.8,
            'aircraft': 0.99,
        'silence': 0.99,
    }

# only applies to main label
config['labels_thresholds'] = {
    'default': 0.5,
    'biophony': 0.9,
    'insect': 0.5,
    'bird': 0.9,
    'songbirds': 0.12,
    'duck-goose-swan': 0.5,
    'grouse-ptarmigan': 0.5,
    'anthrophony': 0.08,
    'aircraft': 0.5,
    'silence': 0.5,
}

# I DID NOT USE THESE NUMBERS
config['new_data_classes_counts'] = {
    'biophony':10*3,
        'insect': 10*4,
        'bird':10**3,
            'songbirds':10*4,
            'duck-goose-swan': 10*4,
            'grouse-ptarmigan': 10*4,
    'anthrophony': 10*4,
        'aircraft': 10**4,
    'silence': 10*3,
}



import glob
from pathlib import Path

In [122]:
# preds_df = teacher.load_all_rawcsv_as_df(config['reg_loc_per_set'][0],config['merged_out_dir'],config['versiontag'])
# preds_df.reset_index(inplace=True,drop=True)

In [123]:
confident_preds_dict = {}
# config['new_data_classes_counts'].keys()
for label_to_sample in ['aircraft','anthrophony']:
    print("--------",label_to_sample)
    confidence_threshold=config['confidence_thresholds'][label_to_sample]
    confident_preds = teacher.pick_confidents(preds_df, label_to_sample,
                                                     confidence_threshold,
                                                     config['related_cols'],
                                                     config['labels_thresholds'],
                                                    columns_to_exclude=['TIMESTAMP','sum','location','region'])
    confident_preds_dict[label_to_sample] = confident_preds.copy()


-------- aircraft
biophony
90958
bird
90657
songbirds
33693
duck-goose-swan
33511
grouse-ptarmigan
33084
insect
29926
silence
29926
-------- anthrophony
biophony
331420
bird
329988
songbirds
65206
duck-goose-swan
64492
grouse-ptarmigan
63360
insect
51945
silence
51945


In [124]:
# confident_preds_dict['aircraft']

In [125]:

# pd.merge(, confident_preds_dict['aircraft'], how='inner', )

In [126]:
selected_data={}

# sort values by sum so other lables has lower confidence
for k,v in confident_preds_dict.items():
    confident_preds_dict[k]=v.sort_values(by='sum', ascending=True)
    
#select samples
for k,v in confident_preds_dict.items():
    print(k,len(v))
    if k in ['anthrophony', 'aircraft']:
        selected_data[k]=v.iloc[:]
    else:
        selected_data[k]=v.iloc[:2000]


aircraft 29926
anthrophony 51945


In [140]:
only_anthrophony=(confident_preds_dict['anthrophony'][~confident_preds_dict['anthrophony'].isin( confident_preds_dict['aircraft']).all(1)])
only_anthrophony=only_anthrophony.sort_values(by='aircraft', ascending=True)
only_anthrophony=only_anthrophony[only_anthrophony['aircraft']<=0.112]

2009

In [128]:
print('label_1','label_2','common')
label_pairs = list(itertools.combinations(confident_preds_dict.keys(), 2))
for k1,k2 in label_pairs:
    if k1!=k2:
        s1 = pd.merge(selected_data[k1], selected_data[k2], how='inner', )
        print(k1,k2,len(s1))



label_1 label_2 common
aircraft anthrophony 29926


In [141]:
confident_preds_dict['anthrophony']=only_anthrophony

In [40]:
import pandas as pd
selected_data_merged=pd.concat(selected_data.values())
selected_data_merged_unq = selected_data_merged.drop_duplicates()

In [11]:
print('all samples - unique samples = diff')
all_count = len(selected_data_merged)
unq_count = len(selected_data_merged_unq)
print(all_count,unq_count,all_count-unq_count)

all samples - unique samples = diff
19953 13989 5964


In [12]:
file_properties_df = pd.read_pickle(
    '/scratch/enis/data/nna/database/allFields_dataV5.pkl')


In [13]:


new_dataset_csv, not_found_rows = teacher.generate_new_dataset(
    selected_data_merged_unq,
    config['versiontag'],
    config['split_out_path'],
    file_properties_df,
    config['upper_taxo_links'],
    config['dataset_version'],
    excell_label_headers= config['excell_label_headers'],
    labels_thresholds= config['labels_thresholds'],
    dry_run=False,
    excell_labels_2_names=config['excell_labels_2_names'],
    stereo2mono=True,
    overwrite=True,
)


# from copy import deepcopy
# new_dataset_csv_backup = deepcopy(new_dataset_csv)
# not_found_rows_backup = deepcopy(not_found_rows)

# del not_found_rows_backup,new_dataset_csv_backup


# write_csv(config['new_dataset_path'],
#           new_dataset_csv,
#           fieldnames=config['excell_all_headers'])

--------------------------------------
Not Found
3928632 TIMESTAMP           2019-07-20_04:29:52
anthrophony                       0.005
aircraft                          0.003
biophony                              1
bird                                  1
songbirds                         0.147
duck-goose-swan                   0.099
grouse-ptarmigan                   0.08
insect                            0.055
silence                           0.149
sum                               2.538
region                             anwr
location                             48
Name: 3928632, dtype: object
--------------------------------------
Not Found
7108181 TIMESTAMP           2019-05-09_19:59:52
anthrophony                       0.018
aircraft                          0.012
biophony                              1
bird                                  1
songbirds                         0.186
duck-goose-swan                   0.116
grouse-ptarmigan                  0.095
insect           

In [34]:
# new_dataset_csv

In [20]:
len(new_dataset_csv)

13968

In [16]:
from copy import deepcopy
new_dataset_csv_backup = deepcopy(new_dataset_csv)
not_found_rows_backup = deepcopy(not_found_rows)




In [18]:
print(config['new_dataset_path'])
teacher.write_csv(config['new_dataset_path'],
          new_dataset_csv,
          fieldnames=config['excell_all_headers'])

./datasetV2.1.2.csv


In [19]:
!cd /home/enis/projects/nna/src/scripts

/home/enis/projects/nna/src/scripts


In [ ]:
for i,row in enumerate(new_dataset_csv):
    relative_part=row['Clip Path'].relative_to('/scratch/enis/data/nna/labeling/megan/3rk9ayjc-V1/')
    new_file='/scratch/enis/data/nna/labeling/megan/3rk9ayjc-V2'/ relative_part
    new_file.parent.mkdir(exist_ok=True,parents=True)
#     print(row['Clip Path'],'\n',new_file)
#     break
    output=change_sampling_rate(row['Clip Path'],new_file,48000)
#     break
    if i%1000==0:
        print(i)

0
1000
2000
3000


In [58]:
output

PosixPath('/scratch/enis/data/nna/labeling/megan/3rk9ayjc-V2/anwr/48/S4A10252_20190709_120000_54m_40s__54m_50s.wav')

In [21]:
import subprocess


In [56]:
def change_sampling_rate(input_file,output_file,sampling_rate):
    
    cmd = ['ffmpeg', '-i', str(input_file),'-y','-ar', str(sampling_rate), str(output_file)]
    
    proc = subprocess.Popen(cmd,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE)

    o, e = proc.communicate()

    if proc.returncode != 0:
        print("---------")
        print(cmd)
        print('Output: ' + o.decode('ascii'))
        print('Error: ' + e.decode('ascii'))
        return 0
    else:
        return output_file


In [37]:
f='/scratch/enis/data/nna/labeling/megan/3rk9ayjc-V1/anwr/48/S4A10252_20190709_120000_54m_40s__54m_50s.wav'
f2='/scratch/enis/data/nna/labeling/megan/3rk9ayjc-V2/anwr/48/S4A10252_20190709_120000_54m_40s__54m_50s.wav'
cmd=change_sampling_rate(f,f2,48000)

['ffmpeg', '-i', '/scratch/enis/data/nna/labeling/megan/3rk9ayjc-V1/anwr/48/S4A10252_20190709_120000_54m_40s__54m_50s.wav', '-y', '-ar', '48000', '/scratch/enis/data/nna/labeling/megan/3rk9ayjc-V2/anwr/48/S4A10252_20190709_120000_54m_40s__54m_50s.wav']


In [38]:
' '.join(cmd)    

'ffmpeg -i /scratch/enis/data/nna/labeling/megan/3rk9ayjc-V1/anwr/48/S4A10252_20190709_120000_54m_40s__54m_50s.wav -y -ar 48000 /scratch/enis/data/nna/labeling/megan/3rk9ayjc-V2/anwr/48/S4A10252_20190709_120000_54m_40s__54m_50s.wav'

In [31]:
cmd

1

14